In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.build_adjaency_matrix.create_adjaency_matrix_glycoscience import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_glycosciencedb import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs


In [2]:
# B = build_adjacency_matrix(labeled_pdb_dir='glycosciencedb/data_reformulate/')
# B.calculate_all_matrix()

In [3]:
num_test = 270 # 299 * 80% 

In [4]:
# Create = create_graph(data_dir='glycosciencedb/data_reformulate/', 
#                       atom_embed_dir='merged_embed/atom_name_embed.csv',
#                       mono_embed_dir='merged_embed/monosaccharide_embed.csv',
#                       ab_embed_dir='merged_embed/ab_embed.csv',
#                       dl_embed_dir='merged_embed/dl_embed.csv',
#                       pf_embed_dir='merged_embed/pf_embed.csv',
#                       num_test=num_test, seed=97211, use_mono=1, use_ab=1, use_dl=1, use_pf=1)

# g, test_index = Create.create_all_graph()

In [5]:
# save_graphs('glycosciencedb/graph_data/data.bin', g)

g = load_graphs('glycosciencedb/graph_data/data.bin')[0][0]

In [6]:
num_epoch = 5000
lr = 1e-3

atom_dim = 256
mono_dim = 128
ab_dim = 64
dl_dim = 64
pf_dim = 64

In [7]:
in_size = atom_dim + mono_dim + ab_dim + dl_dim + pf_dim
# in_size = atom_dim

hidden_size_1 = int(in_size / 2)
hidden_size_2 = 256
hidden_size_3 = 128
hidden_size_4 = 32

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)
features = g.ndata["feat"]
labels = g.ndata["shift_value"]
# masks = g.ndata['train_mask'], g.ndata['test_mask']


# masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']
# masks = g.ndata['train_mask'], g.ndata['test_mask']

print(features.dtype)
print(labels.dtype)
# model = NMR_GCN(in_size=576, hid_size=[256, 128, 64, 32], out_size=1).to(device)
model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)
# model training

# NMR_prediction = NMR_prediction(results_dir='experimental_data_combined/graph_combined/experimental_results_all_hydrogen.csv',
#                                 model_dir='experimental_data_combined/graph_combined/Model_hydrogen_experiment_training.pt',
#                                num_epoch = num_epoch, 
#                                lr = lr)


NMR_prediction = NMR_prediction(results_dir='transfer_learning_results/2D_glycoscience/no_pretrain/training_carbon.csv',
                                results_dir_test = 'transfer_learning_results/2D_glycoscience/no_pretrain/testing_carbon.csv',
                                model_dir='transfer_learning_results/2D_glycoscience/no_pretrain/Model_Godess_carbon.pt', 
                               num_epoch = num_epoch, 
                               lr = lr)


print("Training...")
NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

# acc = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, print_out=True)
acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))


# acc1 = NMR_prediction.evaluate(g, features, labels, masksc[0], saved_model, save_train=True, save_test = False)

# acc2 = NMR_prediction.evaluate(g, features, labels, masksc[1], saved_model, save_train=False, save_test = True)


# print("train RMSE {:.4f}".format(acc1))
# print("test RMSE {:.4f}".format(acc2))

torch.float32
torch.float32
Training...


  0%|                                         | 10/5000 [00:01<06:36, 12.59it/s]

5279
1231
Epoch 00000 | Loss 6151.0737 | train_RMSE 77.6012 | test_RMSE 77.8427 
5279
1231
Epoch 00001 | Loss 6021.4409 | train_RMSE 76.5821 | test_RMSE 76.8343 
5279
1231
Epoch 00002 | Loss 5864.3599 | train_RMSE 75.0877 | test_RMSE 75.3591 
5279
1231
Epoch 00003 | Loss 5638.1504 | train_RMSE 72.8843 | test_RMSE 73.1844 
5279
1231
Epoch 00004 | Loss 5310.8232 | train_RMSE 69.7110 | test_RMSE 70.0515 
5279
1231
Epoch 00005 | Loss 4860.6313 | train_RMSE 65.2507 | test_RMSE 65.6442 
5279
1231
Epoch 00006 | Loss 4252.1968 | train_RMSE 59.1299 | test_RMSE 59.5875 
5279
1231
Epoch 00007 | Loss 3494.8611 | train_RMSE 50.9763 | test_RMSE 51.4947 
5279
1231
Epoch 00008 | Loss 2597.7495 | train_RMSE 40.7066 | test_RMSE 41.2110 
5279
1231
Epoch 00009 | Loss 1657.1731 | train_RMSE 29.9627 | test_RMSE 30.0800 
5279
1231
Epoch 00010 | Loss 916.7646 | train_RMSE 27.5268 | test_RMSE 26.2906 
5279
1231
Epoch 00011 | Loss 777.1625 | train_RMSE 38.3266 | test_RMSE 36.1600 
5279
1231
Epoch 00012 | Loss 1

  1%|▏                                        | 28/5000 [00:01<02:16, 36.32it/s]

5279
1231
Epoch 00017 | Loss 588.4365 | train_RMSE 25.3725 | test_RMSE 25.0200 
5279
1231
Epoch 00018 | Loss 651.3713 | train_RMSE 27.8199 | test_RMSE 27.6268 
5279
1231
Epoch 00019 | Loss 789.2116 | train_RMSE 29.6530 | test_RMSE 29.5028 
5279
1231
Epoch 00020 | Loss 881.4948 | train_RMSE 30.3829 | test_RMSE 30.2170 
5279
1231
Epoch 00021 | Loss 939.6133 | train_RMSE 29.9529 | test_RMSE 29.7355 
5279
1231
Epoch 00022 | Loss 908.2844 | train_RMSE 28.4954 | test_RMSE 28.1964 
5279
1231
Epoch 00023 | Loss 816.6119 | train_RMSE 26.2682 | test_RMSE 25.8544 
5279
1231
Epoch 00024 | Loss 703.7104 | train_RMSE 23.6946 | test_RMSE 23.1283 
5279
1231
Epoch 00025 | Loss 572.3386 | train_RMSE 21.4627 | test_RMSE 20.7180 
5279
1231
Epoch 00026 | Loss 469.8771 | train_RMSE 20.4088 | test_RMSE 19.5116 
5279
1231
Epoch 00027 | Loss 437.9791 | train_RMSE 20.8976 | test_RMSE 19.9380 
5279
1231
Epoch 00028 | Loss 462.7661 | train_RMSE 22.2313 | test_RMSE 21.2895 
5279
1231
Epoch 00029 | Loss 520.6367 | 

...

5279
1231
Epoch 04977 | Loss 17.4880 | train_RMSE 2.7209 | test_RMSE 4.7789 
5279
1231
Epoch 04978 | Loss 19.3699 | train_RMSE 2.5201 | test_RMSE 4.6361 
5279
1231
Epoch 04979 | Loss 15.7466 | train_RMSE 2.6171 | test_RMSE 4.6675 
5279
1231
Epoch 04980 | Loss 16.5529 | train_RMSE 2.5034 | test_RMSE 4.6339 
5279
1231
Epoch 04981 | Loss 17.7079 | train_RMSE 2.4618 | test_RMSE 4.5937 
5279
1231
Epoch 04982 | Loss 15.9935 | train_RMSE 2.6198 | test_RMSE 4.6664 
5279
1231
Epoch 04983 | Loss 17.0114 | train_RMSE 2.6181 | test_RMSE 4.6535 
5279
1231
Epoch 04984 | Loss 15.6228 | train_RMSE 2.5999 | test_RMSE 4.6445 
5279
1231
Epoch 04985 | Loss 15.2067 | train_RMSE 2.7306 | test_RMSE 4.7415 
5279
1231
Epoch 04986 | Loss 15.9760 | train_RMSE 2.5713 | test_RMSE 4.6495 
5279
1231
Epoch 04987 | Loss 16.4181 | train_RMSE 2.5530 | test_RMSE 4.6535 


100%|██████████████████████████████████████▉| 4994/5000 [01:45<00:00, 51.57it/s]

5279
1231
Epoch 04988 | Loss 16.0908 | train_RMSE 2.6187 | test_RMSE 4.7332 
5279
1231
Epoch 04989 | Loss 15.9505 | train_RMSE 2.5392 | test_RMSE 4.6939 
5279
1231
Epoch 04990 | Loss 17.1273 | train_RMSE 2.7900 | test_RMSE 4.7909 
5279
1231
Epoch 04991 | Loss 19.0749 | train_RMSE 2.7811 | test_RMSE 4.8351 
5279
1231
Epoch 04992 | Loss 17.1127 | train_RMSE 2.5789 | test_RMSE 4.7211 
5279
1231
Epoch 04993 | Loss 15.7901 | train_RMSE 3.0606 | test_RMSE 5.0014 
5279
1231
Epoch 04994 | Loss 19.0251 | train_RMSE 2.8266 | test_RMSE 4.9228 
5279
1231
Epoch 04995 | Loss 19.1551 | train_RMSE 2.5238 | test_RMSE 4.7630 
5279
1231
Epoch 04996 | Loss 16.3418 | train_RMSE 2.7881 | test_RMSE 4.8855 
5279
1231
Epoch 04997 | Loss 18.9216 | train_RMSE 2.6611 | test_RMSE 4.7835 
5279
1231
Epoch 04998 | Loss 17.6973 | train_RMSE 2.6037 | test_RMSE 4.7269 


100%|███████████████████████████████████████| 5000/5000 [01:45<00:00, 47.19it/s]


5279
1231
Epoch 04999 | Loss 15.1878 | train_RMSE 3.1380 | test_RMSE 5.0269 
best loss: 100000
Testing...
1231
5279
train RMSE 2.3642
test RMSE 4.6216


In [9]:
# num_epoch = 5000
no_fine_tune = np.array([4.6210, 4.7313, 4.867, 4.4850, 4.9071])
np.mean(no_fine_tune), np.std(no_fine_tune)

(4.72228, 0.15604044860227728)

In [10]:
# num_epoch = 1000
no_fine_tune = np.array([4.8463, 4.7466, 4.7363, 4.8890, 4.8950])
np.mean(no_fine_tune), np.std(no_fine_tune)

(4.82264, 0.06846326314162947)